<a href="https://colab.research.google.com/github/ldselvera/huggingface/blob/main/04-Audio-Models/Audio_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import transformers, torch, torchaudio
import librosa
print(transformers.__version__)
print(torch.__version__)
print(torchaudio.__version__)

4.42.4
2.3.1+cu121
2.3.1+cu121


In [5]:
from transformers import AutoFeatureExtractor, ASTForAudioClassification

In [6]:
audio_path = 'example.mp3'
y, sr = librosa.load(audio_path, sr=None)

In [7]:
# https://huggingface.co/MIT/ast-finetuned-audioset-10-10-0.4593
feature_extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [13]:
# Model not trained for 48,000 sample rate
# result = feature_extractor(y, sampling_rate=sr)

# Pipeline downsamples
result = feature_extractor(y, return_tensors='pt')
result

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


{'input_values': tensor([[[ 0.0770, -0.2676,  0.1092,  ..., -1.2776, -1.2776, -1.2776],
         [ 0.0846, -0.2771,  0.0997,  ..., -1.2776, -1.2776, -1.2776],
         [-0.2939, -0.3674,  0.0095,  ..., -1.2776, -1.2776, -1.2776],
         ...,
         [ 0.2184, -0.0845,  0.2923,  ..., -1.2776, -1.2776, -1.2776],
         [ 0.1963, -0.1293,  0.2475,  ..., -1.2776, -1.2776, -1.2776],
         [-0.0509, -0.4521, -0.0752,  ..., -1.2776, -1.2776, -1.2776]]])}

In [14]:
model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

config.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [15]:
result['input_values']

tensor([[[ 0.0770, -0.2676,  0.1092,  ..., -1.2776, -1.2776, -1.2776],
         [ 0.0846, -0.2771,  0.0997,  ..., -1.2776, -1.2776, -1.2776],
         [-0.2939, -0.3674,  0.0095,  ..., -1.2776, -1.2776, -1.2776],
         ...,
         [ 0.2184, -0.0845,  0.2923,  ..., -1.2776, -1.2776, -1.2776],
         [ 0.1963, -0.1293,  0.2475,  ..., -1.2776, -1.2776, -1.2776],
         [-0.0509, -0.4521, -0.0752,  ..., -1.2776, -1.2776, -1.2776]]])

In [17]:
prediction_logits = model(result['input_values']).logits
prediction_logits

tensor([[ -5.9081,  -9.2253,  -9.4442,  -9.4990, -10.3686,  -9.8169, -10.5372,
          -9.9140, -11.0119, -11.1294, -10.8732, -11.3651, -12.3890, -11.9322,
         -10.1947, -10.5203, -10.9214, -11.4615, -11.2874, -10.9701, -11.3812,
         -11.1792, -10.5879, -10.7083,  -9.4158, -11.2141, -10.9959,  -8.1345,
          -8.3063, -12.1890,  -8.9141,  -8.1373,  -9.5586, -10.1567, -10.2660,
         -11.6801, -10.3482, -10.8475, -10.4001, -10.8535, -11.0698,  -9.5788,
         -12.0086, -10.2645, -10.1869, -10.8037, -10.6147, -10.8702, -11.1633,
         -11.6032, -12.3197, -11.4073, -10.7273, -10.6266, -10.7189, -10.2813,
         -12.3984, -10.6907, -11.4477, -10.8104, -12.4599, -10.8371, -11.6523,
         -11.1756,  -8.7801,  -9.1707, -11.2029, -10.2863, -11.8267,  -9.5817,
         -10.8891, -11.8293,  -7.8608,  -8.4752,  -9.2617, -10.5787, -10.6426,
         -10.5211,  -9.6369, -11.0325,  -9.8193,  -9.8407, -11.3109, -10.0102,
          -9.1413, -11.3183,  -9.6363, -10.5181, -10

In [18]:
predicted_class_ids = prediction_logits.argmax(-1).item()
predicted_class_ids

137

In [20]:
model.config.id2label[predicted_class_ids]

'Music'

### Using pipeline

In [21]:
from transformers import pipeline

In [22]:
pipe = pipeline("audio-classification", model="MIT/ast-finetuned-audioset-10-10-0.4593")

In [23]:
pipe(audio_path)

/usr/local/lib/python3.10/dist-packages/transformers/models/audio_spectrogram_transformer/feature_extraction_audio_spectrogram_transformer.py:118: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  waveform = torch.from_numpy(waveform).unsqueeze(0)


[{'score': 0.48486781120300293, 'label': 'Music'},
 {'score': 0.19131098687648773, 'label': 'Violin, fiddle'},
 {'score': 0.08519725501537323, 'label': 'Musical instrument'},
 {'score': 0.04692428186535835, 'label': 'Bowed string instrument'},
 {'score': 0.045360978692770004, 'label': 'Orchestra'}]